In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time as tm

In [2]:
#Reading the current main directory
main_directory = os.getcwd()

In [3]:
#Rreading the name of the subfoldeer
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]

In [4]:
#Counting number of files on subfolder
list = os.listdir(subfolder_name[1]) 
number_files = len(list)

In [5]:
# Changing directory to the folder
os.chdir(subfolder_with_path[1])

In [6]:
#Using a loop to process all files and populate the main data-frame

# Starting process timming
start = tm.time()

# Creating main data-frame
df_files_level = [] #This array will contain data frames from each file as elements

i = 0
while i < number_files:
    
    ##############################################
    # Processing each file
    ##############################################
    # Saving file's name
    file = list[i]
    
    # Reading number and names of sheets
    df_full = pd.read_excel(file, sheet_name = None)
    sheets_names = df_full.keys()
    number_of_sheets = len(sheets_names)
    
    df_sheets_level = [" "]*number_of_sheets #This array will contain data frames from each sheets as elements
    
    # Reading files
    ThreeCT_data = pd.ExcelFile(file)

    # Saving group's name
    group_name = file[0:2]

    for j in range(0, number_of_sheets):
        
        ##############################################
        # Processing each sheet
        ##############################################
        
        # Reading sheets within files
        df_sheets_level[j] = ThreeCT_data.parse(j,header = None)

        #Dropping the frist 13 rows and the row between the last reading and the "sum"
        rows_to_drop = np.arange(13)
        df_sheets_level[j] = df_sheets_level[j].drop(df_sheets_level[j].index[rows_to_drop])


        #locating empty and juv columns
        e = df_sheets_level[j].loc[13].where(df_sheets_level[j].loc[13].str.contains('mpty')).dropna(how='all')
        e.index #To ge tthe column value
        empty = e.index[0]
        s = df_sheets_level[j].loc[13].where(df_sheets_level[j].loc[13].str.contains('Juv')).dropna(how='all')
        s.index #To getthe column value
        iso = s.index[0]

        # Categorizing "duration" labels wether iso or empty
        df_sheets_level[j].iloc[1:2, empty:empty+1] = 'Duration_empty_(s)'
        df_sheets_level[j].iloc[1:2, iso:iso+1] = 'Duration_juv_(s)'

        #Dropping 13th row, now the 0th row
        df_sheets_level[j] =  df_sheets_level[j].drop( df_sheets_level[j].index[0])

        #Creating and assigning new header
        new_header = df_sheets_level[j].iloc[0] #grab the first row for the header
        df_sheets_level[j] = df_sheets_level[j][1:] #take the data less the header row
        df_sheets_level[j].columns = new_header #set the header row as the df header
        
        # Selecting columns of av verlocity and duration for iso and empty
        df_sheets_level[j] = df_sheets_level[j][["Avg. velocity (cm/s)","Track length (cm)", "Duration_empty_(s)","Duration_juv_(s)"]]
        df_sheets_level[j].columns=["AVG_VEL_cm/s","LOCOMOTION_cm", "DURATION_EMPTY_s", "DURATION_JUV_s"]
        df_sheets_level[j].reset_index(drop=True, inplace=True)

        #Removing the Nan row between the last reading a the 'sum'
        df_sheets_level[j] = df_sheets_level[j].dropna(how='all')

        # Creating time-column vector based on measurements' size
        unit = 20/(len(df_sheets_level[j])-1)
        TIME_minutes = np.arange(unit,21,unit)
        TIME_minutes = np.append(TIME_minutes, "SUM")


        # Adding time in minutes' column
        df_sheets_level[j].insert(0, 'TIME_STEP_minutes', TIME_minutes)
        
        #Adding mouse name
        df_sheets_level[j].insert(0, 'MOUSE_NAME', [group_name + '_' + str(j+1)]*len(df_sheets_level[j]))

        #Summations a diffrences of duration (empty vs iso)
        summ = df_sheets_level[j]["DURATION_EMPTY_s"] + df_sheets_level[j]["DURATION_JUV_s"]
        diff = df_sheets_level[j]["DURATION_JUV_s"] - df_sheets_level[j]['DURATION_EMPTY_s']
        
        # Changing any sum value from zero to 10^20
        summ.loc[summ == 0] = pow(10,20)
        
        # Calculating relative and normalized socialindexes
        df_sheets_level[j]['SOCIAL_INTERACTION_RATIO'] = diff/summ
        df_sheets_level[j]['NORMALIZED_SOCIAL_INTERACTION_RATIO'] = diff/(5*60)

    ################################################################### # Finish of processing sheets
    df_sheets_level = pd.concat(df_sheets_level)
    df_files_level.append(df_sheets_level)

    i += 1
    
################################################################################### # Finish of processing files
appended_data = pd.concat(df_files_level)
end = tm.time()

# Priting running time and files processed
print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 



6  Files processed 
 
 Execution time: 1.61 seconds


In [7]:
#Extract the column of "SUM" for locomotion
sum_column_locomotion = pd.pivot_table(appended_data, values='LOCOMOTION_cm', index=['MOUSE_NAME'],
                    columns=['TIME_STEP_minutes'], aggfunc=np.sum)
sum_column_locomotion = sum_column_locomotion['SUM']

In [8]:
#Extract the column of "SUM" for social interaction
sum_column_social = pd.pivot_table(appended_data, values='SOCIAL_INTERACTION_RATIO', index=['MOUSE_NAME'],
                    columns=['TIME_STEP_minutes'], aggfunc=np.sum)
sum_column_social = sum_column_social['SUM']

In [10]:
#Changing the time step vector to numeric to obtain a properly sorted table
appended_data['TIME_STEP_minutes']= pd.to_numeric(appended_data['TIME_STEP_minutes'], errors='coerce')

#Creating the horizontal table pivoting from the vertical dataframe
table = pd.pivot_table(appended_data, values='SOCIAL_INTERACTION_RATIO', index=['MOUSE_NAME'],
                       columns=['TIME_STEP_minutes'], aggfunc=np.sum)
#Adding the 'SUM' social column
table['SUM'] = sum_column_social

#Adding the 'SUM' locomotion column
table['LOCOMOTION_cm'] = sum_column_locomotion

# exporting table to Excel
table.to_excel('3CT_data_horizontal_format.xlsx')


In [ ]:
# Going back to the main directory
os.chdir(main_directory)